In [7]:
import pyspark

In [8]:
from pyspark.sql import SparkSession
from pyspark import SparkContext

In [26]:
# sc = SparkContext().getOrCreate()

In [ ]:
spark = SparkSession.builder.appName('Basicspyspark').getOrCreate()

In [ ]:
df = spark.createDataFrame([(1, 4), (2, 5), (3, 6)], ["A", "B"])

In [ ]:
df.count()

In [ ]:
df.show()

In [ ]:
# Returns Column Object
df.A

In [10]:
df.select('A').show()

+---+
|  A|
+---+
|  1|
|  2|
|  3|
+---+



In [11]:
# Adding a new column
df.withColumn('C', df.A+1).show()

+---+---+---+
|  A|  B|  C|
+---+---+---+
|  1|  4|  2|
|  2|  5|  3|
|  3|  6|  4|
+---+---+---+



In [12]:
from pyspark.sql.functions import lit
df.withColumn('C', lit(5)).show()

+---+---+---+
|  A|  B|  C|
+---+---+---+
|  1|  4|  5|
|  2|  5|  5|
|  3|  6|  5|
+---+---+---+



In [13]:
df = df.withColumn('C', lit(5))

In [14]:
df.show()

+---+---+---+
|  A|  B|  C|
+---+---+---+
|  1|  4|  5|
|  2|  5|  5|
|  3|  6|  5|
+---+---+---+



In [15]:
df.select('A', (df.A>2).alias('State')).show()

+---+-----+
|  A|State|
+---+-----+
|  1|false|
|  2|false|
|  3| true|
+---+-----+



In [16]:
df[(df.A>2)].show()

+---+---+---+
|  A|  B|  C|
+---+---+---+
|  3|  6|  5|
+---+---+---+



### GroupBy

In [17]:
df = spark.createDataFrame([('a', 33), ('b', 11), ('a', 22)], ['names', 'age'])

In [18]:
gdf = df.groupBy(df.names)

In [19]:
gdf.agg({"*":"count"}).collect()

[Row(names='b', count(1)=1), Row(names='a', count(1)=2)]

In [20]:
from pyspark.sql import functions as F

In [21]:
df = spark.createDataFrame([('a', 33), ('b', 11), ('a', 22)], ['names', 'age'])
gdf = df.groupBy(df.names)

sorted(gdf.agg(F.min(df.age)).collect())

[Row(names='a', min(age)=22), Row(names='b', min(age)=11)]

In [22]:
g2df = df.groupBy(df.names)
g2df.min('age').collect()

[Row(names='b', min(age)=11), Row(names='a', min(age)=22)]

In [35]:
# spark.stop()

### Generate your own DataFrame
Create stringRDD RDD and then convert it into a DataFrame when we're reading stringJSONRDD using spark.read.json.

In [24]:
# Generate our own JSON data 
stringJSONRDD = sc.parallelize((""" 
  { "id": "123",
    "name": "Katie",
    "age": 19,
    "eyeColor": "brown"
  }""",
   """{
    "id": "234",
    "name": "Michael",
    "age": 22,
    "eyeColor": "green"
  }""", 
  """{
    "id": "345",
    "name": "Simone",
    "age": 23,
    "eyeColor": "blue"
  }""")
)

In [25]:
# Create DataFrame
swimmersJSON = spark.read.json(stringJSONRDD)